# Basin Hopping subroutine comparison

Using the so-called EggHolder test function, the following code compares over $1000$ samples:

1) the classical basin-hopping method (with uniform displacement as subroutine) run for $100$ steps

2) the monotonic basin-hopping method (with uniform displacement as subroutine) run for $100$ steps

3) the basin-hopping method with skipping as subroutine run for $100$ steps

In [1]:
import numpy as np
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import itertools
import warnings
import logging
import os
import time
import math

import scipy
from scipy import optimize
from scipy.optimize import basinhopping
from scipy.stats import gengamma

def monotonicityconstraintQ(x, function, threshold):
    # checks only threshold value constraints
    return function(x) < threshold

def boundaryconstraintQ(x, boxsize):
    # checks only boundary constraints
    return bool(np.all(x <= boxsize)) and bool(np.all(x >= -boxsize))

def bothconstraintsQ(x, boxsize, function, threshold):
    # checks for both boundary and threshold value constraints
    return boundaryconstraintQ(x, boxsize) and monotonicityconstraintQ(x, function, threshold)

def bothconstraintsQindependent(x, boxsize, function, threshold):
    # checks for both boundary and threshold value constraints independently
    return boundaryconstraintQ(x, boxsize), monotonicityconstraintQ(x, function, threshold)

def basinQ(x):
    if (x[1]>350 and 15*x[1]-13*x[0]+150<=1):
        return True
    else:
        return False
    
def checkcoordinates(x, boxsize):
    newx = x
    for i in range(x.shape[0]):
        if x[i] > boxsize:
            newx[i] = x[i] - 2*boxsize
        elif x[i] < -boxsize:
            newx[i] = x[i] + 2*boxsize
    return newx

class Uniform(object):
    
    def __init__(self, boxsize, stepsize=1, periodicboundary=False, proposeoutsidebounds=False):
        self.stepsize = stepsize
        self.boxsize = boxsize
        self.periodicboundary = periodicboundary
        self.proposeoutsidebounds = proposeoutsidebounds
        
    def __call__(self, x):
        sigma = self.stepsize
        boxsize = self.boxsize
        periodicboundary = self.periodicboundary
        proposeoutsidebounds = self.proposeoutsidebounds
        
        global emptymoves
        
        d = x.shape[0]
        xnew = x + np.random.uniform(low=-sigma, high=sigma, size=d)
        
        if (proposeoutsidebounds or boundaryconstraintQ(xnew, boxsize)):
            return xnew
        elif periodicboundary:
            xnew = checkcoordinates(xnew, boxsize)
            return xnew
        else:
            emptymoves += 1
            return x


class MonotonicRWM(object):
    
    def __init__(self, function, boxsize, stepsize=1, periodicboundary=False, proposeoutsidebounds=False):
        self.stepsize = stepsize
        self.boxsize = boxsize
        self.periodicboundary = periodicboundary
        self.proposeoutsidebounds = proposeoutsidebounds
        self.function = function
        
    def __call__(self, x):
        sigma = self.stepsize
        boxsize = self.boxsize
        periodicboundary = self.periodicboundary
        proposeoutsidebounds = self.proposeoutsidebounds
        function = self.function
        global emptymoves
        
        d = x.shape[0]
        fval = function(x)
        
        xnew = x + np.random.uniform(low=-sigma, high=sigma, size=d)
        monotonicityflag = monotonicityconstraintQ(xnew, function, fval)
        
        if (proposeoutsidebounds or boundaryconstraintQ(xnew, boxsize)):
            if monotonicityflag:
                return xnew
            else:
                emptymoves += 1
                return x
        elif periodicboundary:
            xnew = checkcoordinates(xnew, boxsize)
            if monotonicityflag:
                return xnew
            else:
                emptymoves += 1
                return x
        else:
            emptymoves += 1
            return x

class GaussianRWTakeStep(object):
    
    def __init__(self, boxsize, stepsize=1,  periodicboundary=False, proposeoutsidebounds=False):
        self.stepsize = stepsize
        self.boxsize = boxsize
        self.periodicboundary = periodicboundary
        self.proposeoutsidebounds = proposeoutsidebounds
        
    def __call__(self, x):
        sigma = self.stepsize
        boxsize = self.boxsize
        periodicboundary = self.periodicboundary
        proposeoutsidebounds = self.proposeoutsidebounds

        global emptymoves
        d = x.shape[0]
        
        phi = np.random.normal(loc=0.0, scale=sigma, size=d)
        phi = phi/np.linalg.norm(phi)
        beta = math.sqrt(2*sigma)
        r = gengamma.rvs(a = d/2., c = 2., loc = 0., scale = beta, size = 1)
        xnew = x + phi*r[0]
        
        if proposeoutsidebounds or boundaryconstraintQ(xnew, boxsize):
            return xnew
        elif periodicboundary:
            xnew = checkcoordinates(xnew, boxsize)
            return xnew
        else:
            emptymoves += 1
            return x        
        
class MonotonicSkippingTakeStep(object):
    
    def __init__(self, function, boxsize, maxjumps=50, stepsize=1, periodicboundary=False, proposeoutsidebounds=False):
        self.function = function
        self.maxjumps = maxjumps
        self.stepsize = stepsize
        self.boxsize = boxsize
        self.periodicboundary = periodicboundary
        self.proposeoutsidebounds = proposeoutsidebounds
        
    def __call__(self, x):
        sigma = self.stepsize
        maxjumps = self.maxjumps
        boxsize = self.boxsize
        periodicboundary = self.periodicboundary
        proposeoutsidebounds = self.proposeoutsidebounds
        function = self.function
        global sss
        global arraynfval
        global emptymoves

        d = x.shape[0]
        nfval = 1
        fval = function(x)
        
        phi = np.random.normal(loc=0.0, scale=sigma, size=d)
        phi = phi/np.linalg.norm(phi)
        beta = math.sqrt(2*sigma)
        r = gengamma.rvs(a = d/2., c = 2., loc = 0., scale = beta, size = maxjumps)
        xnew = x + phi*r[0]
        j = 1
        
        if proposeoutsidebounds:
            xnew = checkcoordinates(xnew, boxsize)
            monotonicityflag = monotonicityconstraintQ(xnew, function, fval)
            nfval += 1
            while (j < maxjumps and not(monotonicityflag)):
                xnew += phi*r[j]
                j += 1
                monotonicityflag = monotonicityconstraintQ(xnew, function, fval)
                nfval += 1
            
            arraynfval.append(nfval)
            
            if j>1 and j < maxjumps and monotonicityflag:
                sss += 1

            if monotonicityflag:
                return xnew
            else:
                emptymoves += 1
                return x
            
        elif periodicboundary:
            xnew = checkcoordinates(xnew, boxsize)
            monotonicityflag = monotonicityconstraintQ(xnew, function, fval)
            nfval += 1
            while (j < maxjumps and not(monotonicityflag)):
                xnew += phi*r[j]
                xnew = checkcoordinates(xnew, boxsize)
                j += 1
                monotonicityflag = monotonicityconstraintQ(xnew, function, fval)
                nfval += 1
                
            arraynfval.append(nfval)
            
            if j>1 and j < maxjumps and monotonicityflag:
                sss += 1
     
            if monotonicityflag:
                return xnew
            else:
                emptymoves += 1
                return x
            
        else:
            boundaryflag, monotonicityflag = bothconstraintsQindependent(xnew, boxsize, function, fval)
            nfval += 1
            while (j < maxjumps and boundaryflag and not(monotonicityflag)):
                xnew += phi*r[j]
                j += 1
                boundaryflag, monotonicityflag = bothconstraintsQindependent(xnew, boxsize, function, fval)
                nfval += 1
                
            arraynfval.append(nfval)
        
            if j>1 and j < maxjumps and boundaryflag and monotonicityflag:
                sss += 1
        
            if boundaryflag and monotonicityflag:
                return xnew
            else:
                emptymoves += 1
                return x
    
class MyBounds(object):
    
    def __init__(self, box):
        self.xmax = np.array(box)
        self.xmin = np.array(-box)
        
    def __call__(self, **kwargs):
        x = kwargs["x_new"]
        return bool(np.all(x <= self.xmax)) and bool(np.all(x >= self.xmin))
    
def interleave(arrays, axis=0, out=None):
    shape = list(np.asanyarray(arrays[0]).shape)
    if axis < 0:
        axis += len(shape)
    assert 0 <= axis < len(shape), "'axis' is out of bounds"
    if out is not None:
        out = out.reshape(shape[:axis+1] + [len(arrays)] + shape[axis+1:])
    shape[axis] = -1
    return np.stack(arrays, axis=axis+1, out=out).reshape(shape)

def show_doubletrajectory(startingpoints, proposedpoints, x, boxsize):
    margin = boxsize * 1.05
    sp = np.array(startingpoints)
    pp = np.array(proposedpoints)
    tj = interleave((sp,pp), axis=0)
    tj = np.vstack([tj,x])
    plt.figure()
    plt.xlim(-margin,margin)
    plt.ylim(-margin,margin)
    plt.plot(tj[:, 0], tj[:, 1], '-o', sp[:, 0], sp[:, 1], 's', x[0], x[1], 'r*') 
    plt.gca().set_aspect('equal', adjustable='box') 
    return plt.show()
    
def show_trajectory(trajectory, x, boxsize):
    margin = boxsize * 1.05
    tj = np.array(trajectory)
    tj = np.vstack([tj,x])
    plt.figure()
    plt.xlim(-margin,margin)
    plt.ylim(-margin,margin)
    plt.plot(tj[:, 0], tj[:, 1], '-o', x[0], x[1], 'r*') 
    plt.gca().set_aspect('equal', adjustable='box') 
    return plt.show()

def eggholder(x):
    # self.global_optimum = [[512,404.2319]]
    # self.fglob = -959.6407
    # self._bounds = [(-512.0, 512.0), (-512.0, 512.0)]
    return -(x[1]+ 47.)* np.sin(np.sqrt(np.abs(x[1] + x[0]/2. + 47.))) - x[0]*np.sin(np.sqrt(np.abs(x[0] - x[1] - 47.)))

def BHcomparison(function, n_samples = 50, maxsteps = 100, T = 2.0, stepsize = 2.0, maxjumps = 200, maxlocalnonexits = 50, periodicboundary = True, boxsize = 100, optmethod = 'L-BFGS-B', seed = 1, plotflag = True):
    global emptymoves
    global arraynfval
    global sss
    
    trueoptimum = np.array([512, 404.2319])
    truevalue = function(trueoptimum)

    nfeval_bh = np.zeros(n_samples, dtype=np.int64)
    nfeval_bhu = np.zeros(n_samples, dtype=np.int64)
    nfeval_mbh = np.zeros(n_samples, dtype=np.int64)
    nfeval_bhs = np.zeros(n_samples, dtype=np.int64)
    
    gap_bh = np.zeros(n_samples)
    gap_bhu = np.zeros(n_samples)
    gap_mbh = np.zeros(n_samples)
    gap_bhs = np.zeros(n_samples)
    
    dist_bh = np.zeros(n_samples)
    dist_bhu = np.zeros(n_samples)
    dist_mbh = np.zeros(n_samples)
    dist_bhs = np.zeros(n_samples)
    
    sol_bh = np.zeros([n_samples,2])
    sol_bhu = np.zeros([n_samples,2])
    sol_mbh = np.zeros([n_samples,2])
    sol_bhs = np.zeros([n_samples,2])
    
    basin_bh = np.zeros(n_samples)    
    basin_bhu = np.zeros(n_samples)
    basin_mbh = np.zeros(n_samples)
    basin_bhs = np.zeros(n_samples)
    
    afterbasin_bh = np.zeros(n_samples)
    afterbasin_bhu = np.zeros(n_samples)
    afterbasin_mbh = np.zeros(n_samples)
    afterbasin_bhs = np.zeros(n_samples)
    
    skippingjumpszt = np.zeros(n_samples)
    
    xmin = [-boxsize, -boxsize]
    xmax = [boxsize, boxsize]
    bounds = [(low, high) for low, high in zip(xmin, xmax)] #bounds for local minimizer (rewritten in the way required by L-BFGS-B, which can be used as the problem is smooth and bounded)
    minimizer_kwargs = dict(bounds=bounds, method=optmethod)
    mybounds = MyBounds(np.array([boxsize]*2)) #bounds for accept_test

    np.random.seed(seed)
    ip = np.random.uniform(low=-boxsize, high=boxsize, size=[n_samples,2])
    
    print('Standard BH T=',T)
    emptymoves = 0
    startg = time.time()
    start = time.time()
    tweakedstepsize = math.sqrt(3)*stepsize
    for i in range(n_samples):
        arraynfval = []
        x = ip[i]
        res_bh = basinhopping(function, x, niter=maxsteps, T=T, stepsize=tweakedstepsize, minimizer_kwargs=minimizer_kwargs, interval=1000, accept_test=mybounds, seed=seed)
        sol_bh[i] = res_bh.x
        nfeval_bh[i] = np.sum(arraynfval) + res_bh.nfev
        dist_bh[i] = np.linalg.norm(res_bh.x - trueoptimum)
        gap_bh[i] = res_bh.fun - truevalue
    end = time.time()
    print('Average #empty moves =',emptymoves/(maxsteps*n_samples))
    print('Execution time={:.4f} s'.format(end-start),' nfeval={:0.0f}'.format(np.sum(nfeval_bh)),'\nAvg nfeval/run={:0.0f}'.format(np.average(nfeval_bh)), " Median={:.3f}".format(np.median(nfeval_bh)),"2.5 Percentile={:.3f}".format(np.percentile(nfeval_bh,2.5)),"97.5 Percentile={:.3f}".format(np.percentile(nfeval_bh,97.5)))
    
    mytakestep = Uniform(boxsize=boxsize, stepsize=stepsize, periodicboundary=True, proposeoutsidebounds=False)

    print('\nUniform BH T=',T)
    emptymoves = 0
    startg = time.time()
    start = time.time()
    tweakedstepsize = math.sqrt(3)*stepsize
    for i in range(n_samples):
        arraynfval = []
        x = ip[i]
        res_bhu = basinhopping(function, x, niter=maxsteps, T=T, stepsize=tweakedstepsize, minimizer_kwargs=minimizer_kwargs, interval=1000, take_step=mytakestep, accept_test=mybounds, seed=seed)
        sol_bhu[i] = res_bhu.x
        nfeval_bhu[i] = np.sum(arraynfval) + res_bhu.nfev
        dist_bhu[i] = np.linalg.norm(res_bhu.x - trueoptimum)
        gap_bhu[i] = res_bhu.fun - truevalue
    end = time.time()
    print('Average #empty moves =',emptymoves/(maxsteps*n_samples))
    print('Execution time={:.4f} s'.format(end-start),' nfeval={:0.0f}'.format(np.sum(nfeval_bhu)),'\nAvg nfeval/run={:0.0f}'.format(np.average(nfeval_bhu)), " Median={:.3f}".format(np.median(nfeval_bhu)),"2.5 Percentile={:.3f}".format(np.percentile(nfeval_bhu,2.5)),"97.5 Percentile={:.3f}".format(np.percentile(nfeval_bhu,97.5)))

    mytakestep = MonotonicRWM(function=function, boxsize=boxsize, stepsize=tweakedstepsize, periodicboundary=True, proposeoutsidebounds=False)
    
    print('\nMonotonic BH')
    start = time.time()
    emptymoves = 0
    for i in range(n_samples):
        arraynfval = []
        x = ip[i]
        res_mbh = basinhopping(function, x, niter=maxsteps, T=T, minimizer_kwargs=minimizer_kwargs, interval=1000, take_step=mytakestep, accept_test=mybounds, seed=seed)
        sol_mbh[i] = res_mbh.x
        nfeval_mbh[i] = np.sum(arraynfval) + res_mbh.nfev
        dist_mbh[i] = np.linalg.norm(res_mbh.x - trueoptimum)
        gap_mbh[i] = res_mbh.fun - truevalue
    end = time.time()
    print('Average #empty moves =',emptymoves/(maxsteps*n_samples))
    print('Execution time={:.4f} s'.format(end-start),' nfeval={:0.0f}'.format(np.sum(nfeval_mbh)),'\nAvg nfeval/run={:0.0f}'.format(np.average(nfeval_mbh)), " Median={:.3f}".format(np.median(nfeval_mbh)),"2.5 Percentile={:.3f}".format(np.percentile(nfeval_mbh,2.5)),"97.5 Percentile={:.3f}".format(np.percentile(nfeval_mbh,97.5)))
    
    mytakestep = MonotonicSkippingTakeStep(function=function, maxjumps=maxjumps, boxsize=boxsize, stepsize=stepsize, periodicboundary=True, proposeoutsidebounds=False)
    
    print('\nBH with Skipping')
    start = time.time()
    emptymoves = 0
    for i in range(n_samples):
        arraynfval = []
        sss = 0
        x = ip[i]
        res_bhs = basinhopping(function, x, niter=maxsteps, T=T, minimizer_kwargs=minimizer_kwargs, interval=1000, take_step=mytakestep, accept_test=mybounds, seed=seed)
        sol_bhs[i] = res_bhs.x
        nfeval_bhs[i] = np.sum(arraynfval) + res_bhs.nfev
        dist_bhs[i] = np.linalg.norm(res_bhs.x - trueoptimum)
        gap_bhs[i] = res_bhs.fun - truevalue
        skippingjumpszt[i] = sss
    end = time.time()
    print('Average #empty moves =',emptymoves/(maxsteps*n_samples))
    print('Average #jumps={:.1f}'.format(np.sum(skippingjumpszt)/n_samples),' true average #jumps={:.1f}'.format(np.sum(skippingjumpszt[skippingjumpszt.nonzero()])/len(skippingjumpszt[skippingjumpszt.nonzero()])))
    print('Execution time={:.4f} s'.format(end-start),' nfeval={:0.0f}'.format(np.sum(nfeval_bhs)),'\nAvg nfeval/run={:0.0f}'.format(np.average(nfeval_bhs)), " Median={:.3f}".format(np.median(nfeval_bhs)),"2.5 Percentile={:.3f}".format(np.percentile(nfeval_bhs,2.5)),"97.5 Percentile={:.3f}".format(np.percentile(nfeval_bhs,97.5)))

    print('Global execution time={:.4f} s\n'.format(time.time()-startg))

    for i in range(n_samples):
        afterbasin_bh[i] = basinQ(sol_bh[i])
        afterbasin_bhu[i] = basinQ(sol_bhu[i])
        afterbasin_mbh[i] = basinQ(sol_mbh[i])
        afterbasin_bhs[i] = basinQ(sol_bhs[i])

    print("Fraction of points in the basin of the global minimum for uniform BH:    {:.3f}".format(np.average(afterbasin_bhu)))
    print("Fraction of points in the basin of the global minimum for standard BH:   {:.3f}".format(np.average(afterbasin_bh)))
    print("Fraction of points in the basin of the global minimum for MBH:           {:.3f}".format(np.average(afterbasin_mbh)))
    print("Fraction of points in the basin of the global minimum for BHS:           {:.3f}".format(np.average(afterbasin_bhs)))
    
    print("\nEuclidean distance from global minimum for uniform BH: \nAvg={:.2f}".format(np.average(dist_bhu)), " Median={:.3f}".format(np.median(dist_bhu)),"2.5 Percentile={:.3f}".format(np.percentile(dist_bhu,2.5)),"97.5 Percentile={:.3f}".format(np.percentile(dist_bhu,97.5)))
    print("\nEuclidean distance from global minimum for standard BH: \nAvg={:.2f}".format(np.average(dist_bh)), " Median={:.3f}".format(np.median(dist_bh)),"2.5 Percentile={:.3f}".format(np.percentile(dist_bh,2.5)),"97.5 Percentile={:.3f}".format(np.percentile(dist_bh,97.5)))
    print("\nEuclidean distance from global minimum for MBH:\nAvg={:.2f}".format(np.average(dist_mbh)), " Median={:.3f}".format(np.median(dist_mbh)),"2.5 Percentile={:.3f}".format(np.percentile(dist_mbh,2.5)),"97.5 Percentile={:.3f}".format(np.percentile(dist_mbh,97.5)))
    print("\nEuclidean distance from global minimum for BH-S:\nAvg={:.2f}".format(np.average(dist_bhs))," Median={:.3f}".format(np.median(dist_bhs)),"2.5 Percentile={:.3f}".format(np.percentile(dist_bhs,2.5)),"97.5 Percentile={:.3f}".format(np.percentile(dist_bhs,97.5)))
    

    print("\n\nGap from global minimum value for uniform BH:\nAvg={:.2f}".format(np.average(gap_bhu)), " Median={:.3f}".format(np.median(gap_bhu)),"2.5 Percentile={:.3f}".format(np.percentile(gap_bhu,2.5)),"97.5 Percentile={:.3f}".format(np.percentile(gap_bhu,97.5)))
    print("\nGap from global minimum value for standard BH:\nAvg={:.2f}".format(np.average(gap_bh)), " Median={:.3f}".format(np.median(gap_bh)),"2.5 Percentile={:.3f}".format(np.percentile(gap_bh,2.5)),"97.5 Percentile={:.3f}".format(np.percentile(gap_bh,97.5)))
    print("\nGap from global minimum value for MBH:\nAvg={:.2f}".format(np.average(gap_mbh)), " Median={:.3f}".format(np.median(gap_mbh)),"2.5 Percentile={:.3f}".format(np.percentile(gap_mbh,2.5)),"97.5 Percentile={:.3f}".format(np.percentile(gap_mbh,97.5)))
    print("\nGap from global minimum value for BH-S:\nAvg={:.2f}".format(np.average(gap_bhs)), " Median={:.3f}".format(np.median(gap_bhs)),"2.5 Percentile={:.3f}".format(np.percentile(gap_bhs,2.5)),"97.5 Percentile={:.3f}".format(np.percentile(gap_bhs,97.5)))
    
    return None

In [2]:
BHcomparison(function=eggholder, n_samples = 1000, maxsteps = 100, T = 1.0, stepsize = 1.0, maxjumps = 200, maxlocalnonexits = 50, periodicboundary = True, boxsize = 512, optmethod = 'L-BFGS-B', seed = 1, plotflag = True)

Standard BH T= 1.0
Average #empty moves = 0.0
Execution time=122.5462 s  nfeval=2095659 
Avg nfeval/run=2096  Median=2010.000 2.5 Percentile=1571.925 97.5 Percentile=3090.000

Uniform BH T= 1.0
Average #empty moves = 0.0
Execution time=123.0747 s  nfeval=2100060 
Avg nfeval/run=2100  Median=2004.000 2.5 Percentile=1775.925 97.5 Percentile=3432.375

Monotonic BH
Average #empty moves = 0.9636
Execution time=65.2276 s  nfeval=860652 
Avg nfeval/run=861  Median=348.000 2.5 Percentile=324.000 97.5 Percentile=5085.000

BH with Skipping
Average #empty moves = 0.96155
Average #jumps=3.8  true average #jumps=3.9
Execution time=332.9528 s  nfeval=20728499 
Avg nfeval/run=20728  Median=20370.000 2.5 Percentile=19956.975 97.5 Percentile=23702.275
Global execution time=521.2582 s

Fraction of points in the basin of the global minimum for uniform BH:    0.022
Fraction of points in the basin of the global minimum for standard BH:   0.022
Fraction of points in the basin of the global minimum for MBH: 